In [ ]:
"""
@Authors:                   @Email:
* Shaiful                     shaiful019@gmail.com
* Rezwanul                    rezwanul.cse@gmail.com
* Oli
* Habibul                     shovon2111@gmail.com
* Tahmid
"""

# Problem: Return to Origin (RTO) Customers Prediction
 Problem class: Supervised Classification
 
 Problem dataset link: Real dataset so no online link.
 
 Problem description: 
 
     RTO (Return to Origin) is a Classification Problem in eCommerce platforms where Many customers cancel their order when the   product is already on shipping. Then customer don't response and the product is return back to office.
 
 Problem Task:
     Have to predict who will Cancel the product order after product is already on shipping.

# Importing libraries

In [8]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

# Feature or column description

COLUMN DESCRIPTIONS:
* Order Id
* User Id
* Order Value (In BDT, amount of the order)
* Payment Type (How, customers paid for it. MPD/MPC = Manual, means Cash on delivery. MPS = Bkash, Rocket type mobile payments. * OPS = online credit card, debit card. ADC = AD Cash (can be earned, can be recharged), EMI = Monthly EMI Purchase )
* District (Where customers are getting delivered)
* OrderSource (Android, Mobile Site or Desktop Site)
* Order Type for Delivery ( Express = Quick delivery and paid more for it, Normal = average delivery)
* Delivery Charge ( for the delivery)
* Order Placing Date (When customer placed the order)
* Order Confirmation date (Office make calls to confirm the order. It call is made late, they might not want the product later)
* IsCartOrder (is it a cart order or single product order)
* OrderFromPromotionalEvent (Is product purchased in discount from a promotional event or not)
* courierService (Courier service used by the customer)
* isRTO ( Order canceled after product is already on shipping )

# Load Dataset & Merging the datasets

In [9]:
df_1 = pd.read_csv("dataset/RTOdata.csv")
df_2 = pd.read_csv("dataset/RTO_data_new.csv")

# Merging
df = pd.concat([df_1, df_2], ignore_index=True)
# type(df_1)
# type(df_2)
df.shape

(20000, 14)

In [10]:
# df.head()
df.tail()

,OrderId,UserId,OrderValue,PaymentType,District,OrderSource,OrderType,DeliveryCharge,OrderPlacedDay,OrderConfirmDayOverPhone,IsCartOrder,OrderFromPromotionalEvent,courierService,isRTO
19995,2610588,901210,187,MPD,Nilphamari,android,normal,60.0,7/17/2019 19:29,7/17/2019 13:33,No,Yes,Bangladesh Postal Service ...,Yes
19996,2610586,528636,235,MPS,Sylhet City,android,normal,0.0,7/17/2019 19:29,7/18/2019 5:55,No,Yes,E-Desh ...,No
19997,2610583,649604,170,MPS,Bagerhat,android,normal,25.0,7/17/2019 19:27,7/17/2019 15:42,No,Yes,Paperfly ...,No
19998,2610576,967577,409,MPD,Dhaka,android,normal,25.0,7/17/2019 19:20,7/17/2019 13:28,No,No,E-Desh ...,No
19999,2610572,970326,402,MPD,Moulvibazar,android,normal,60.0,7/17/2019 19:18,7/17/2019 13:45,No,Yes,Paperfly ...,Yes


### Drop Unnecessary Columns

In [11]:
df.drop("OrderId", axis=1, inplace=True)
df.tail()

,UserId,OrderValue,PaymentType,District,OrderSource,OrderType,DeliveryCharge,OrderPlacedDay,OrderConfirmDayOverPhone,IsCartOrder,OrderFromPromotionalEvent,courierService,isRTO
19995,901210,187,MPD,Nilphamari,android,normal,60.0,7/17/2019 19:29,7/17/2019 13:33,No,Yes,Bangladesh Postal Service ...,Yes
19996,528636,235,MPS,Sylhet City,android,normal,0.0,7/17/2019 19:29,7/18/2019 5:55,No,Yes,E-Desh ...,No
19997,649604,170,MPS,Bagerhat,android,normal,25.0,7/17/2019 19:27,7/17/2019 15:42,No,Yes,Paperfly ...,No
19998,967577,409,MPD,Dhaka,android,normal,25.0,7/17/2019 19:20,7/17/2019 13:28,No,No,E-Desh ...,No
19999,970326,402,MPD,Moulvibazar,android,normal,60.0,7/17/2019 19:18,7/17/2019 13:45,No,Yes,Paperfly ...,Yes
